In [1]:
import os

import pandas as pd
import numpy as np

In [2]:
os.chdir('../90_Original_Data/')

In [3]:
pd.read_csv('products.csv').head(3) # prod_df

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7


In [4]:
pd.read_csv('orders.csv').head(3) # ord_df

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0


In [5]:
pd.read_csv('departments.csv').head(3) # dep_df

,department_id,department
0,1,frozen
1,2,other
2,3,bakery


In [6]:
pd.read_csv('aisles.csv').head(3) # ais_df

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars


In [9]:
pd.read_csv('order_products__prior.csv').head(3) # ord_prod_train_df

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0


In [8]:
pd.read_csv('order_products__train.csv').head(3) # ord_prod_test_df

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0


Here we're combining the order/product information with user information by merging the `order_products` table with the `orders` table.

In [13]:
pd.read_csv('order_products__train.csv')\
        .merge(pd.read_csv('orders.csv'),
        on='order_id').head(3) # ord_prod_train_df

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,train,4,4,10,9.0
1,1,11109,2,1,112108,train,4,4,10,9.0
2,1,10246,3,0,112108,train,4,4,10,9.0


In [24]:
train_carts = (ord_prod_test_df.groupby('user_id',as_index=False)
                                      .agg({'product_id':(lambda x: set(x))})
                                      .rename(columns={'product_id':'latest_cart'}))

df = df.merge(train_carts, on='user_id')
df['in_cart'] = (df.apply(lambda row: row['product_id'] in row['latest_cart'], axis=1).astype(int))

,product_id,user_id,user_product_total_orders,latest_cart,in_cart
0,1,138,2,{42475},0
1,907,138,2,{42475},0
2,1000,138,1,{42475},0
3,3265,138,1,{42475},0
4,4913,138,1,{42475},0


In [31]:
df

,product_id,user_id,user_product_total_orders,latest_cart,in_cart
0,1,138,2,{42475},0
1,907,138,2,{42475},0
2,1000,138,1,{42475},0
3,3265,138,1,{42475},0
4,4913,138,1,{42475},0
5,6184,138,1,{42475},0
6,6901,138,1,{42475},0
7,7147,138,1,{42475},0
8,7987,138,1,{42475},0
9,8048,138,2,{42475},0


In [26]:
df.in_cart.value_counts()

0    7645837
1     828824
Name: in_cart, dtype: int64

In [27]:
df.user_id.nunique()

131209

In [ ]:
target_pcts = df.in_cart.value_counts(normalize=True) 
print(target_pcts)

target_pcts.plot(kind='bar')